# Tutorial 1

other doc sources:
https://github.com/MarcToussaint/rai-maintenance/tree/master/help

In [1]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/vasko/Documents/TUB3/AI_Robotics/robotics-course/tutorials'

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}




## Setting up a basic Config

The starting point is to create a `Configuration`.

In [2]:
C = ry.Config()
D = C.view()

This shows an empty configuration. Tip: Make the view window appear "Always On Top" (right click on the window bar)

You can add things (objects, scene models, robots) to a configuration.

In [3]:
C.clear()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')

Note that the view was updated automatically. (Internally, the view 'subscribes' to updates of the shared memory of Config).

In [4]:
ball = C.addFrame(name="ball")
ball.setShape(ry.ST.sphere, [.1])
ball.setPosition([.8,.8,1.5])
ball.setColor([1,1,0])

One can also add convex meshes (just passing the vertex array), or use sphere-swept convex meshes (ssBox, capsule, sphere, etc)

In [5]:
hand = C.addFrame(name="hand", parent="pr2L")
hand.setShape(ry.ST.ssBox, size=[.2,.2,.1,.02]),
hand.setRelativePosition([0,0,-.1])
hand.setColor([1,1,0])

In this last example, the new object has another frame (pr2L) as *parent*. This means that it is permanently attached to this parent. pos and quat/rot are interpreted relative to the parent.

## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the $n\times 7$ matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedoms (DOFs). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection* onto the actual DOFs

In [31]:
q = C.getJointState()
print('joint names: ', C.getJointNames() )
print('joint state: ', q)

joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'l_shoulder_pan_joint', 'l_shoulder_lift_joint', 'l_upper_arm_roll_joint', 'l_elbow_flex_joint', 'l_forearm_roll_joint', 'l_wrist_flex_joint', 'l_wrist_roll_joint']
joint state:  [ 2.          1.          2.28762102  1.00001     0.50001     1.00001
 -2.00001     1.50001    -0.50001     0.50001   ]


In [24]:
q[2]

8.570796326794897

Let's move the configuration by adding to the joint configuration

In [25]:
q[0] = q[0] + 1.
C.setJointState(q)

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [26]:
X0 = C.getFrameState()
print('frame state: ', X0)

frame state:  [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 2.          1.          0.         ...  0.          0.
  -0.91021607]
 [ 2.03284933  0.96230489  0.890675   ... -0.64361996  0.29283673
  -0.64361996]
 ...
 [ 0.5         2.          0.4        ...  0.          0.
   0.        ]
 [ 0.8         0.8         1.5        ...  0.          0.
   0.        ]
 [ 1.51353223  1.32435024  0.96711955 ...  0.0029326   0.77729817
  -0.61627576]]


Let's do a questionable thing: adding .1 to all numbers in the frame matrix!

In [27]:
X = X0 + .1
C.setFrameState(X)

The rows of X have non-normalized quaternions! These are normalized when setting the frame state.

Also, the frame poses are now *inconsistent* to the joint constraints! We can read out the projected joint state, set the joint state, and get a consistent state again:

In [28]:
C.setJointState( C.getJointState() )

Let's bring the configuration back into the state before the harsh *setFrame*

In [29]:
C.setFrameState(X0)

 ## Selecting joints

Often one would like to choose which joints are actually active, that is, which joints are referred to in q. This allows one to sub-select joints and work only with projections of the full configuration state. This changes the joint state dimensionality, including ordering of entries in q.

The frame state is not affected by such a selection of active joints.

In [33]:
C.selectJointsByTag(["armL","base"])
# C.selectJointsByTag([])
q = C.getJointState()
print('joint state: ', q)
print('joint names: ', C.getJointNames() )

joint state:  [ 2.          1.          2.28762102  1.00001     0.50001     1.00001
 -2.00001     1.50001    -0.50001     0.50001   ]
joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'l_shoulder_pan_joint', 'l_shoulder_lift_joint', 'l_upper_arm_roll_joint', 'l_elbow_flex_joint', 'l_forearm_roll_joint', 'l_wrist_flex_joint', 'l_wrist_roll_joint']


## Features & Jacobians

A core part of rai defines features over configurations. A feature is a differentiable mapping from a configuration (or set of configurations) to a vector. Starndard features are "position-of-endeffector-X" or "distance/penetration-between-convex-shapes-A-and-B", etc. But there are many, many more features defined in rai, like error of Newton-Euler-equations for an object, total energy of the system, etc. Defining differentiable features is the core of many functionalities in the rai code.

Let's define a basic feature over C: the 3D (world coordinate) position of pr2L (left hand)

In [34]:
F = C.feature(ry.FS.position, ["pr2L"])

We can now evaluate the feature, and also get the Jacobian:

In [35]:
print(F.description(C))

[y,J] = F.eval(C)
print('hand position =', y)
print('Jacobian =', J)

F_Position/0-pr2L
hand position = [1.49350457 1.22861844 0.94627935]
Jacobian = [[ 1.          0.         -0.37386935 -0.3106267  -0.15575831 -0.13479104
  -0.5072611  -0.16950556  0.06671492 -0.01884874]
 [ 0.          1.         -0.67330771 -0.3582334  -0.08144115 -0.0421164
   0.00352249 -0.02194985 -0.05672006  0.01443724]
 [ 0.          0.         -0.03744408  0.29027279 -0.24437351 -0.41773446
   0.09321806  0.07372961  0.19643616 -0.04820521]]


Another example

In [36]:
F2 = C.feature(ry.FS.distance, ["hand", "ball"])
print(F2.description(C))

F_PairCollision/0-hand-ball


In [37]:
F2.eval(C)

(array([-0.8028018]),
 array([[-0.69591852, -0.53016574,  0.69076829,  0.63622891,  0.10739423,
         -0.13890609,  0.49209686,  0.20338689,  0.1411892 , -0.04720631]]))

## Camera views (needs more testing)

We can also add a frame, attached to the head, which has no shape associated to it, but create a view is associated with that frame:

In [38]:
C.addFrame(name='camera', parent='head_tilt_link', args='Q:<d(-90 1 0 0) d(180 0 0 1)> focalLength:.3')
V = C.cameraView()
IV = V.imageViewer()
V.addSensor(name='camera', frameAttached='camera', width=600, height=400)

In [39]:
[image,depth] = V.computeImageAndDepth()

When we move the robot, that view moves with it:

In [40]:
C.setJointState(q=np.asarray([0.5]), joints=['head_pan_joint'])
V.updateConfig(C)
V.computeImageAndDepth()

(array([[[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[129, 140, 151],
         [129, 140, 151],
         [129, 140, 151],
         ...,
         [129, 140, 151],
         [ 73,  73,  73],
         [ 73,  73,  73]],
 
        ...,
 
        [[120, 120, 120],
         [120, 120, 120],
         [121, 121, 121],
         ...,
         [218, 218,   5],
         [218, 218,   5],
         [218, 218,   5]],
 
        [[121, 121, 121],
         [126, 126, 126],
         [126, 126, 126],
         ...,
         [218, 218,   5],
         [218, 218,   5],
         [218, 218,   5]],
 
        [[126, 126, 126],
         [126, 126, 126],
         [126, 126, 126],
         ...,
         [218, 218,   5],
  

To close a view (or destroy a handle to a computational module), we reassign it to zero. We can also remove a frame from the configuration.

In [41]:
IV = 0
V = 0
C.delFrame('camera')